In [1]:
import pandas as pd
import os

# excel stuff doesn't work without this:
import sys
sys.path.append("C:/Users/pmallick/anaconda3/Lib/site-packages")
# if someone else is using this replace the above path with the base dir of 'openpyxl'.

In [2]:
# read in main csv file
signups = pd.read_excel(os.path.join("data_input", "signups.xlsx"))

# filter by done/not done
signups = signups.loc[pd.isna(signups['Emailed'])]
signups.reset_index(drop = True, inplace=True)

# remove irrelevant/already done requests
signups = signups[["Ready",
                   "First Name",
                   "Last Name",
                   "Email",
                   "Phone",
                   "Are you part of a volunteer group?",
                   "Group Name",
                   "Count",
                   "Comments / Feedback"]]

# strip dangling spaces in all columns
signups.columns = signups.columns.str.strip()

#convert postcard requests to int 
signups['Count'] = signups.Count.astype(int)

# remove duplicate email rows
signups.drop_duplicates(subset=["Email"], keep='first', inplace=True)
signups.reset_index(drop=True, inplace=True)

signups

,Ready,First Name,Last Name,Email,Phone,Are you part of a volunteer group?,Group Name,Count,Comments / Feedback
0,NaN,Linda,Miller,canuck0345@yahoo.com,6268482934,Yes,Indivisible,100,Thank you for providing the opportunity to help!
1,NaN,Michael,Griffin,438elder@gmail.com,9092915848,NaN,NaN,25,NaN
2,NaN,Carol,Hannon,ceegeeshops@yahoo.com,951-505-9075,Yes,Indivisible 41,25,NaN
3,NaN,Shelley,McFarland,s2mcfarland4@gmail.com,5304006028,Yes,NaN,50,I am doing these postcards on my own (have wri...
4,NaN,Jennifer,Goodall,Djgoodall2@gmail.com,2164082936,NaN,NaN,25,NaN


In [3]:
import numpy as np
flagged_indices = []


if (not os.path.exists("data_output")):
    os.mkdir("data_output")

# save flagged rows
signup_flags = signups.iloc[flagged_indices]
if os.path.isfile("data_output/flags.xlsx"):
    os.remove("data_output/flags.xlsx")



# save rows that have comments
signup_comments = signups.loc[signups['Comments / Feedback'].notna()]
if os.path.isfile("data_output/comments.xlsx"):
    os.remove("data_output/comments.xlsx")

# save groups too bc y not
signup_groups = signups.loc[signups['Group Name'].notna()]
if os.path.isfile("data_output/groups.xlsx"):
    os.remove("data_output/groups.xlsx")

# save all as excel
signup_flags.to_excel(os.path.join("data_output", "flags.xlsx"), index=False)
signup_comments.to_excel(os.path.join("data_output", "comments.xlsx"), index=False)
signup_groups.to_excel(os.path.join("data_output", "groups.xlsx"), index=False)

# remove from signups dataframe
signups.drop(flagged_indices, inplace = True)
signups.reset_index(drop=True, inplace=True)

print("Invalid postcard number: ", len(signup_flags), " rows")

Invalid postcard number:  0  rows


In [4]:
# final preprocessing signups:
signups = signups[["First Name", "Last Name", "Email", "Count"]]
signups

,First Name,Last Name,Email,Count
0,Linda,Miller,canuck0345@yahoo.com,100
1,Michael,Griffin,438elder@gmail.com,25
2,Carol,Hannon,ceegeeshops@yahoo.com,25
3,Shelley,McFarland,s2mcfarland4@gmail.com,50
4,Jennifer,Goodall,Djgoodall2@gmail.com,25


In [5]:
# read in addresses csv file
addresses = pd.read_excel(os.path.join("data_input", "addresses.xlsx"))

# filter by done/not done
addresses = addresses.loc[addresses['Assigned'].isna()]
addresses = addresses[["NAME", "ADDRESS", "CITY STATE ZIP"]]
addresses.reset_index(drop = True, inplace=True)
addresses.insert(0, "Group/Volunteer", "")
addresses.insert(0, "Assigned", "")

print("length: ", len(addresses))
display(addresses.head())

length:  3050


,Assigned,Group/Volunteer,NAME,ADDRESS,CITY STATE ZIP
0,,,John & Pamela Hartman,12105 Beaufait Ave,"Northridge, CA 91326"
1,,,Horovtioun Samarghachian,12120 Beaufait Ave,"Northridge, CA 91326"
2,,,Siegel Family,12130 Beaufait Ave,"Northridge, CA 91326"
3,,,Guillermina Trevis,12602 Henzie Pl,"Granada Hills, CA 91344"
4,,,Christine Ogtanyan,12200 Beaufait Ave,"Northridge, CA 91326"


In [6]:
# compute how many people can be mailed

running_card_num = 0
total_card_num = len(addresses)
total_needed = sum(signups["Count"])

breaked = False

for row in signups.iterrows():
    idx = row[0]
    row = row[1]
    num_cards = row["Count"]
    running_card_num += num_cards
    # if sum > total, then go back an index and break --> go back bc this one is too much
    if (running_card_num > total_card_num):
        idx -=1
        breaked = True
        break;

print("total addresses needed: ", total_needed)
print("we have: ", total_card_num)
if total_needed <= total_card_num:
    print("leftover: ", total_card_num - total_needed)

if breaked:
    print("Not enough names for all volunteers. Only ", idx+1, " will be sent.")

total addresses needed:  225
we have:  3050
leftover:  2825


In [7]:
# sort
if not os.path.exists(os.path.join("data_output", "sorted_addresses")):
    os.mkdir(os.path.join("data_output", "sorted_addresses"))

import os
curr_index = 0
for row in signups.iterrows():
    idx = row[0]
    row = row[1]
    fname = row['First Name']
    lname = row['Last Name']
    email = row['Email']
    fullname = fname.strip() + " " + lname.strip()
    num_cards = row['Count']
    if curr_index + num_cards > len(addresses):
        print("Not enough addresses. Stopping.")
        break;
        
    personCards = addresses.iloc[curr_index: curr_index+num_cards]
    for i in range(num_cards):
        addresses.loc[curr_index, 'Assigned'] = "PM"
        addresses.loc[curr_index, 'Group/Volunteer'] = fullname
        curr_index += 1
    signups.at[idx, "Ready"] = "PM"
    personCards = personCards.drop(columns=['Assigned', "Group/Volunteer"])
    personCards = pd.concat([pd.DataFrame({'Volunteer': [email] }), personCards])
    personCards.to_excel(os.path.join("data_output", "sorted_addresses", fullname + " " + str(num_cards) + ".xlsx"), index=False)
    
    

# replace all ""s in "Assigned" to NaNs
# addresses["Assigned"].loc[addresses['Assigned'] == "", ] = np.nan

addresses["Assigned"].loc[addresses['Assigned'] == ""] = np.nan

addresses.to_excel(os.path.join("data_output", "addresses.xlsx"))
display(addresses)
signups.to_excel(os.path.join("data_output", "signups.xlsx"))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Assigned,Group/Volunteer,NAME,ADDRESS,CITY STATE ZIP
0,PM,Linda Miller,John & Pamela Hartman,12105 Beaufait Ave,"Northridge, CA 91326"
1,PM,Linda Miller,Horovtioun Samarghachian,12120 Beaufait Ave,"Northridge, CA 91326"
2,PM,Linda Miller,Siegel Family,12130 Beaufait Ave,"Northridge, CA 91326"
3,PM,Linda Miller,Guillermina Trevis,12602 Henzie Pl,"Granada Hills, CA 91344"
4,PM,Linda Miller,Christine Ogtanyan,12200 Beaufait Ave,"Northridge, CA 91326"
...,...,...,...,...,...
3045,NaN,,Julie Abrescy or Hong Family,1364 Harold Ave,"Simi Valley, CA 93065"
3046,NaN,,Spencer Wickman,4232 Cochran St,"Simi Valley, CA 93063"
3047,NaN,,Guillermo Ramos,1204 Haven Ave,"Simi Valley, CA 93065"
3048,NaN,,Arsenio Saenz,1205 Haven Ave,"Simi Valley, CA 93065"


In [8]:
signups

,First Name,Last Name,Email,Count,Ready
0,Linda,Miller,canuck0345@yahoo.com,100,PM
1,Michael,Griffin,438elder@gmail.com,25,PM
2,Carol,Hannon,ceegeeshops@yahoo.com,25,PM
3,Shelley,McFarland,s2mcfarland4@gmail.com,50,PM
4,Jennifer,Goodall,Djgoodall2@gmail.com,25,PM
